In [22]:
import os
import datetime
import pandas as pd
import re
import great_expectations
import agriculture as ag

In [23]:
#!pip install great_expectations

## Descargando los datos

Para descargar los datos referentes a los precios de diversos productos agricolas, podemos ejecutar directamente el script `agriculture.py` en la terminal de la siguiente manera:

```python3 agriculture.py```

El script mencionado llama de forma iterativa a la API de la pagina del Sistema Nacional de Informacion de Mercados (SNIIM) de la Secretaria de Economia, para recuperar los precios de diversos productos en diversos centros de distribucion en el pais, desde el año 2020 hasta la fecha.

Otra manera de descargar los datos es ejecutando la siguiente celda, la cual inicializa un objeto de tipo `ScrapperMarketAgriculture`, el cual contiene la funcion `scraping`, la cual realiza el mismo proceso del script mencionado anteriormente.

In [24]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
# scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
# scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Una vez recuperados los datos, estos se almacenan de forma local en archivos .csv, los cuales se encuentran en el directorio `data/sniim`

In [25]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = "./data/sniim/"

## Procesamiento de los datos

Como al final tendremos una dataframe conteniendo los precios de diversos productos, podemos tratar de obtener el nombre de cada producto a partir del nombre del archivo csv, para esto definimos la siguiente funcion:

In [26]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

Finalmente, podemos iterar sobre los archivos presentes en el directorio anterior, para generar una dataframe por cada uno de ellos, agregar las dataframes a una lista para posteriormente concatenarlas en una sola.

Ademas, a cada dataframe le agregamos la columna `producto` para poder diferenciar las entradas una vez concatenadas las dataframes, y separamos la columna `destino` en dos: `destino` y `central` para identificar mas facilmente la Entidad de destino y la central de abastos donde se levanto la muestra.

In [27]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file)) 
        df["producto"] = estandarizar_nombre(file) # Obtenemos el nombre del producto a partir del nombre del archivo
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True) # Separamos la columna destino en 2
        
        # Reordenamos las columnas
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        # Agregamos la dataframe generada a la lista
        df_list.append(df)

In [28]:
precios_sniim = pd.concat(df_list,ignore_index=True) # Concatenamos nuestra lista de dataframes en una sola

Una vez obtenida nuestra dataframe final, podemos utilizar el metodo `sample` para obtener una vista general de nuestra dataframe.

In [29]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
653075,27/08/2020,Ajo Morado,Kilogramo,Michoacán,Michoacán,Mercado de Abasto de Morelia,80.0,90.0,90.00,NaN
1455012,25/02/2022,Toronja Roja,Kilogramo,Jalisco,Colima,Centros de distribución de Colima,14.0,15.0,15.00,NaN
506103,03/03/2022,Cebolla Bola Grande,Kilogramo,Tamaulipas,Zacatecas,Mercado de Abasto de Zacatecas,24.0,24.0,24.00,NaN
847821,16/01/2020,Calabacita Italiana,Caja de 11 kg.,Puebla,Jalisco,Mercado Felipe Ángeles de Guadalajara,27.27,27.27,27.27,SELECCIONADA
1455324,30/12/2022,Toronja Roja,Caja de 25 kg.,Nuevo León,Chihuahua,Central de Abasto de Chihuahua,22.4,30.0,23.20,NaN
784954,20/06/2023,Calabaza De Castilla,Kilogramo,Nuevo León,Tamaulipas,Módulo de Abasto de Reynosa,15.0,18.0,18.00,NaN
1380389,24/01/2022,Nopal,Caja de 20 kg.,México,San Luis Potosí,Centro de Abasto de San Luis Potosí,10.0,10.5,10.00,NaN
1682875,31/05/2022,Pera D´Anjou #100,Caja de 20 kg.,Importación,Durango,Centro de Distribución y Abasto de Gómez Palacio,40.0,45.0,42.00,NaN
1174545,25/02/2022,Tomate Verde,Caja de 25 kg.,Distrito Federal,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",12.8,14.0,12.80,NaN
1464352,21/03/2023,Platano Chiapas Calidad Exportacion,Caja de 18 kg.,Chiapas,DF,Central de Abasto de Iztapalapa DF,15.0,17.22,16.67,NaN


Para una mayor facilidad al momento de comprender nuestra dataframe, generamos un diccionario de datos para esta, explicando lo que representa cada una de las columnas.

In [30]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra (MXN)",
    "precio_max": "El valor más alto de la cotización dentro de una muestra (MXN)",
    "precio_frec": "Es el dato que más se repite en la muestra (moda) (MXN)",
    "obs": "Observaciones encontradas para la muestra"
}

In [31]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df

,0
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (mo...
obs,Observaciones encontradas para la muestra


## Limpieza de los datos

Una vez generado nuestro diccionario, podemos comenzar a tratar los datos de nuestro dataframe. Algo que podemos observar del sample de nuestro dataframe es que la columna `obs` parece ser la unica que cuenta con valores nulos. Podemos tener una idea de la proporcion de datos faltantes en esta columna ejecutando la siguiente celda, la cual calcula el porcentaje de entradas faltantes en la misma.

In [32]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

Porcentaje de entradas faltantes en columna 'obs': 95.78%


Vemos que la mayoria de las entradas en nuestra dataframe (> 90%) no cuenta con informacion en la columna `obs`, por lo que puede ser mas conveniente deshacernos de esta columna.

In [33]:
precios_sniim.drop(columns="obs", inplace=True)

In [34]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
902342,25/10/2022,Sandia Jubilee,Kilogramo,Colima,Colima,Centros de distribución de Colima,18.0,20.0,20.00
276023,23/03/2020,Chile Jalapeño,Kilogramo,Sinaloa,Morelos,Central de Abasto de Cuautla,22.0,25.0,22.00
1586784,05/10/2023,Piña Chica,Pieza,Veracruz,Yucatán,Centro Mayorista Oxkutzcab,30.0,30.0,30.00
1497071,25/10/2021,Nopal,Caja de 20 kg.,Distrito Federal,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",14.0,15.0,15.00
1644429,31/03/2021,Manzana Red Delicious,Caja de 17 kg.,Importación,Jalisco,Mercado de Abasto de Guadalajara,38.24,42.94,38.24
906506,08/08/2022,Papaya Maradol,Kilogramo,Oaxaca,Hidalgo,Central de Abasto de Pachuca,10.0,11.0,11.00
690099,06/04/2021,Elote Mediano,Docena,Campeche,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",2.08,2.5,2.08
75832,16/04/2020,Chile Guajillo,Kilogramo,Zacatecas,Jalisco,Mercado Felipe Ángeles de Guadalajara,72.0,72.0,72.00
1252390,20/09/2022,Tomate Saladette,Caja de 13 kg.,Sinaloa,Jalisco,Mercado de Abasto de Guadalajara,16.15,16.92,16.15
1495922,07/02/2020,Papa Alpha,Kilogramo,Nayarit,Nayarit,Nayarabastos de Tepic,15.0,16.5,16.00


Posteriormente, podemos utilizar el metodo `info` de nuestra dataframe para obtener informacion acerca de los tipos de datos de las columnas o la posible presencia de mas datos faltantes.

In [35]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703507 entries, 0 to 1703506
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   fecha         object 
 1   producto      object 
 2   presentacion  object 
 3   origen        object 
 4   destino       object 
 5   central       object 
 6   precio_min    object 
 7   precio_max    object 
 8   precio_frec   float64
dtypes: float64(1), object(8)
memory usage: 117.0+ MB


Lo primero que notamos es que la columna `fecha` tiene un tipo de dato `object`, lo cual puede significar que esta almacenada como un string. Al tratarse de datos temporales, podemos convertir esta columna a un objeto de tipo `datetime` para una mayor facilidad al momento de manipular o explorar los datos.

In [36]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"])

Despues podemos analizar los valores en la columna `origen` para verificar que no haya valores inesperados. Los posibles valores en esta columna son los nombres de los 32 estados de Mexico, o bien "Importacion" si el producto fue importado del extranjero, o "Nacional" si no hay datos sobre su estado de origen.

In [37]:
precios_sniim["origen"].unique()

array(['Nayarit', 'Distrito Federal', 'Oaxaca', 'Importación', 'Sonora',
       'Yucatán', 'Baja California', 'Campeche', 'San Luis Potosí',
       'Chiapas', 'Durango', 'Hidalgo', 'Michoacán', 'Jalisco',
       'Guanajuato', 'México', 'Quintana Roo', 'Sinaloa', 'Puebla',
       'Nuevo León', 'Aguascalientes', 'Zacatecas', 'Tamaulipas',
       'Chihuahua', 'Coahuila', 'Guerrero', 'Colima', 'Veracruz',
       'Querétaro', 'Tabasco', 'Morelos', 'Baja California Sur',
       'Nacional'], dtype=object)

Vemos que no existen anomalias aparentes en esta columna, asi que podemos proceder a revisar los valores de la columna `destino`, la cual deberia tener valores similares a la columna anterior.

In [38]:
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California ', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Vemos que tampoco hay anomalias en esta columna. Sin embargo, algunos valores contienen espacios al final del valor (por ejemplo, `Baja California `). Esto se puede corregir facilmente utilizando la funcion `strip` que nos ofrece pandas:

In [39]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Finalmente, la columna `precio_max` aparece como de tipo `object`, por lo que posiblemente hay valores que impidieron que fuera parseada a `float`.

Despues de una inspeccion de esta columna, vemos que para algunos precios, se utiliza la coma (,) para separar los millares. Podemos eliminar esta coma de nuestro dataframe con el metodo `replace`, para despues transformarla a tipo `float` con el metodo `astype(float)`.

In [40]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(str).str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)

In [43]:
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(str).str.replace(",","")
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(float)

Finalmente, podemos volver a obtener una muestra de nuestro dataframe para ver su estructura final, asi como volver a llamar al metodo `info` para verificar los tipos de datos de cada columna y que no haya valores faltantes.

In [44]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
1688016,2020-08-28,Piña Chica,Pieza,Quintana Roo,Yucatán,"Mercado ""Casa del Pueblo""",15.00,15.00,15.00
81201,2021-02-07,Chile Serrano,Arpilla de 25 kg.,Sinaloa,Chihuahua,Central de Abasto de Chihuahua,17.00,18.00,18.00
1635856,2020-12-24,Manzana Golden Delicious,Kilogramo,Chihuahua,Coahuila,"Central de Abasto de La Laguna, Torreón",16.00,17.00,16.50
155874,2022-08-03,Chile Poblano,Kilogramo,Distrito Federal,Guerrero,Central de Abastos de Acapulco,59.00,59.00,59.00
97948,2022-11-18,Zanahoria Polvo,Arpilla de 23 kg.,Guanajuato,Jalisco,Mercado de Abasto de Guadalajara,3.91,4.35,3.91
818786,2023-08-03,Chicharo Arrugado,Kilogramo,México,Michoacán,Mercado de Abasto de Morelia,35.00,38.00,38.00
1470890,2023-10-01,Platano Chiapas,Caja de 18 kg.,Chiapas,Morelos,Central de Abasto de Cuautla,11.11,16.67,13.89
563466,2021-10-19,Fresa,Caja de 8 kg.,Michoacán,Jalisco,Mercado Felipe Ángeles de Guadalajara,43.13,43.13,43.13
870373,2020-04-30,Lechuga Romanita Grande,Pieza,San Luis Potosí,San Luis Potosí,Centro de Abasto de San Luis Potosí,7.00,8.00,7.00
1040865,2022-08-18,Limon Con Semilla #5,Kilogramo,Michoacán,Guanajuato,Central de Abasto de León,20.00,22.00,20.00


In [45]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703507 entries, 0 to 1703506
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   fecha         datetime64[ns]
 1   producto      object        
 2   presentacion  object        
 3   origen        object        
 4   destino       object        
 5   central       object        
 6   precio_min    float64       
 7   precio_max    float64       
 8   precio_frec   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 117.0+ MB
